<a href="https://colab.research.google.com/github/aziz122596/Gentex/blob/main/%D0%A0%D0%B0%D0%BA_%D1%8F%D0%B8%D1%87%D0%BD%D0%B8%D0%BA%D0%BE%D0%B2_16_10.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Описание кода для классификации пациентов на основе данных белков**

Этот код реализует полный процесс анализа данных о белках здоровых и больных пациентов, включая их предобработку, импутацию пропущенных значений, обучение модели классификации и оценку ее качества. Модель достигает точности 94.59% на тестовом наборе данных. Ниже приведено описание каждого шага кода:

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.impute import KNNImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix

## **1. Загрузка всех данных о белках и аннотации**

- **file_path**: Путь к файлу Excel с данными.
- Загрузка данных о здоровых и больных пациентах из соответствующих листов ('health' и 'disease').
- Загрузка аннотации колонок из листа 'аннотация колонок', чтобы получить дополнительную информацию о типе образцов.

In [ ]:
file_path = 'merged_datasets.xlsx'

In [ ]:
df_healthy = pd.read_excel(file_path, sheet_name='health')
df_disease = pd.read_excel(file_path, sheet_name='disease')

In [ ]:
df_annotation = pd.read_excel(file_path, sheet_name='аннотация колонок')

In [ ]:
available_proteins = df_disease['Accession'].dropna().unique()

In [ ]:
df_disease_filtered_all = df_disease[df_disease['Accession'].isin(available_proteins)]

## **2. Фильтрация доступных белков**

- Из данных выбираются только те белки, которые присутствуют в обоих наборах данных (здоровые и больные).
- Данные фильтруются по этим доступным белкам для обоих типов пациентов.

In [ ]:
df_disease_filtered_all = df_disease_filtered_all.set_index('Accession').T
df_disease_filtered_all.columns.name = None
df_disease_filtered_all.reset_index(drop=True, inplace=True)

In [ ]:
df_healthy_filtered_all = df_healthy[df_healthy['Accession'].isin(available_proteins)]

In [ ]:
df_healthy_filtered_all = df_healthy_filtered_all.set_index('Accession').T
df_healthy_filtered_all.columns.name = None
df_healthy_filtered_all.reset_index(drop=True, inplace=True)

## **3. Объединение данных о здоровых и больных пациентах**

- Данные о больных пациентах транспонируются и добавляется метка состояния здоровья `Health_Status = 1` (больной).
- Данные о здоровых пациентах также транспонируются и добавляется метка состояния здоровья `Health_Status = 0` (здоров).
- Объединение данных о здоровых и больных пациентах в единый DataFrame.

In [ ]:
df_disease_filtered_all['Health_Status'] = 1  # Метка "больной"
df_healthy_filtered_all['Health_Status'] = 0  # Метка "здоров"

In [ ]:
df_combined_all = pd.concat([df_disease_filtered_all, df_healthy_filtered_all], ignore_index=True)

In [ ]:
for col in df_combined_all.columns[:-1]:  # исключаем колонку Health_Status
    min_value = df_combined_all[col].min(skipna=True)
    df_combined_all[col].fillna(min_value, inplace=True)

## **4. Импутация пропущенных значений**

- Пропуски, которые могут быть нерендомными (например, ниже порога обнаружения), заполняются минимальными значениями для каждого белка.
- Оставшиеся пропуски заполняются с использованием KNN-импутации, что учитывает зависимость между признаками и помогает корректнее заполнить значения.

In [ ]:
imputer = KNNImputer(n_neighbors=5)
X_imputed = imputer.fit_transform(df_combined_all.drop(columns=['Health_Status']))

In [ ]:
df_combined_imputed_all = pd.DataFrame(X_imputed)

In [ ]:
df_combined_imputed_all.columns = [f'Protein_{i}' for i in range(df_combined_imputed_all.shape[1])]

In [ ]:
df_combined_imputed_all['Health_Status'] = df_combined_all['Health_Status'].values

## **5. Разделение данных на обучающую и тестовую выборки**

- Данные разделяются на признаки (X) и метки (y).
- Выполняется разделение на обучающую и тестовую выборки с использованием `train_test_split`. 20% данных выделяются для тестирования.

In [ ]:
X = df_combined_imputed_all.drop(columns=['Health_Status'])
y = df_combined_imputed_all['Health_Status']

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42, stratify=y)

## **6. Стандартизация данных**

- Стандартизация признаков выполняется с помощью `StandardScaler`, чтобы обеспечить одинаковый масштаб для всех признаков, что помогает улучшить производительность модели.


In [ ]:
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test

## **7. Обучение модели с использованием RandomForestClassifier**

- Модель случайного леса (`RandomForestClassifier`) обучается на стандартизированных данных.
- Используется 100 деревьев (`n_estimators=100`), что обеспечивает высокую производительность и стабильность модели.

In [ ]:
model_rf = RandomForestClassifier(n_estimators=100, random_state=42)
model_rf.fit(X_train_scaled, y_train)

## **8. Прогнозирование на тестовом наборе**

- Выполняется предсказание классов для тестовой выборки с использованием обученной модели.

In [ ]:
y_pred = model_rf.predict(X_test_scaled)

## **9. Оценка качества модели**

- **Точность модели**: Выводится общая точность модели на тестовом наборе данных, которая составляет **94.59%**.
- **Отчет о классификации**: Включает метрики `precision`, `recall` и `f1-score` для каждого класса (здоров и болен), чтобы дать полное представление о производительности модели.

In [ ]:
accuracy = accuracy_score(y_test, y_pred)
print(f'\nТочность на тестовом наборе: {accuracy:.4f}')
print('\nОтчет о классификации:')
print(classification_report(y_test, y_pred, target_names=['Здоров', 'Болен']))

## **10. Матрица ошибок**

- Строится матрица ошибок с помощью `confusion_matrix` и визуализируется с использованием `seaborn`. Это позволяет наглядно оценить, какие ошибки модель совершала чаще всего.


In [ ]:
cm = confusion_matrix(y_test, y_pred)
sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', xticklabels=['Здоров', 'Болен'], yticklabels=['Здоров', 'Болен'])
plt.xlabel('Предсказанный класс')
plt.ylabel('Истинный класс')
plt.show()

### **Заключение**

- **Точность модели** на тестовом наборе данных составляет 94.59%, что указывает на высокую производительность модели при классификации пациентов.
- **Матрица ошибок** показывает, что только 2 образца были классифицированы неправильно, что является отличным результатом.
- Импутация пропущенных значений и использование всех доступных белков позволили значительно улучшить качество классификации по сравнению с предыдущими подходами, которые основывались на ограниченном числе белков и пороговых значениях.

Этот подход позволяет более точно различать здоровых и больных пациентов, и использует всю доступную информацию о белках для обучения модели.Также, можно провести дополнительную оптимизацию гиперпараметров или анализ важности признаков для дальнейшего улучшения модели.